In [4]:
import numpy as np
from functools import reduce
import math

def loadDataSet():
    #6条文档=》6个文档向量（32） 6个标签 1个词集（不重复，32）
    postingList= [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                  ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]                                                         #类别标签向量，1代表侮辱性词汇，0代表不是
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

def createVocabList(dataSet):
    vocabSet = set([])                      #创建一个空的不重复列表
    for document in dataSet:               
        vocabSet = vocabSet | set(document) #取并集
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)                                    #创建一个其中所含元素都为0的向量
    
    for word in inputSet:                                                #遍历每个词条
        if word in vocabList:                                            #如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
            # print(returnVec)
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec        

def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)  #6                          #计算训练的文档数目
    # print(numTrainDocs)
    numWords = len(trainMatrix[0])    #向量长度 32
    # print(numWords)
    pAbusive = sum(trainCategory)/float(numTrainDocs)  #文档属于侮辱类的概率 先验概率
    # print(pAbusive)
    p0Num = np.zeros(numWords);    
    p1Num = np.zeros(numWords)    #创建numpy.zeros数组,词条出现数初始化为0
    # print(p1Num)
    p0Denom = 0.0; p1Denom = 0.0   #分母初始化为0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:  #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            # print(p1Num)
            p1Denom += sum(trainMatrix[i])
            # print(p1Denom)
        else:                #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom         
    # print(p1Vect)
    # print(p1Denom)
    p0Vect = p0Num/p0Denom         
    return p0Vect,p1Vect,pAbusive                            #返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率

def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # print(vec2Classify) #测试数据 文档的向量 
    # print(p1Vec) # 训练好的类条件概率 侮辱类单词的类条件概率向量 P( word | 侮辱类)
    # print(vec2Classify * p1Vec) # 侮辱类文档的类条件概率向量 P( docment | 侮辱类)
    # print((vec2Classify * p1Vec).tolist())    
    
    p1c = sum( map(lambda x: math.log(x) if x>0 else 0,(vec2Classify * p1Vec).tolist()))  
    p0c = sum( map(lambda x: math.log(x) if x>0 else 0,(vec2Classify * p0Vec).tolist()))  
    p1 = p1c + math.log(pClass1)    #对应元素相乘
    p0 = p0c + math.log((1.0 - pClass1))
    print('p0:',p0)
    print('p1:',p1)
    if p1 > p0:
        return True
    else: 
        return False

if __name__ == '__main__':
    postingList, classVec = loadDataSet() #文档集 标签集
    # for each in postingLIst:
    #     print(each)
    # print(classVec)
    
    myVocabList = createVocabList(postingList) #词集
    print('myVocabList:\n',myVocabList)
    
    # 文档集 标签集 词集 =》》 trainMat集
    trainMat = []
    for postinDoc in postingList:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    for elment in trainMat:
        print( elment)
    p0V, p1V, pAb = trainNB0(trainMat, classVec) #===> 某个单词的类条件概率    P(WORD|H)    P(H|E) >< P(~H|E)
    print('p0V:\n', p0V)
    print('p1V:\n', p1V)
    print('classVec:\n', classVec)
    print('pAb:\n', pAb)
   
    testEntry = ['love', 'my', 'dalmation']                                    #测试样本1
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))                #测试样本向量化
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')                                        #执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')                                        #执行分类并打印分类结果
    
    testEntry = ['stupid', 'garbage']                                        #测试样本2
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))                #测试样本向量化    
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')                                        #执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')                                        #执行分类并打印分类结果


myVocabList:
 ['take', 'is', 'buying', 'park', 'maybe', 'cute', 'licks', 'has', 'please', 'flea', 'problems', 'steak', 'love', 'how', 'him', 'stupid', 'posting', 'dalmation', 'garbage', 'ate', 'help', 'to', 'I', 'dog', 'my', 'so', 'stop', 'worthless', 'mr', 'quit', 'not', 'food']
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]
p0V:
 [0.         0.04166667 0.         0.         0.         0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.041666